In [0]:
from pyspark.sql.readwriter import DataFrameWriter

spark.conf.set("spark.sql.shuffle.partitions",100)

df_fact_sales_orders = spark.sql("select o.or_id AS Order_Id,o.order_unique_id,o.customer_unique_id,c.c_name AS Customer_Name,c.customer_mobile_number,p.product_id,p.product_name,pay.payment_transaction_id,pay.payment_status,payl.payment_type_value AS payment_type,s.seller_id,s.seller_organization_name,p.product_unit_cost, \
p.product_max_discount_price AS Max_Discount_Amount,coalesce((p.product_unit_cost-p.product_max_discount_price),p.product_unit_cost) AS Final_price_per_unit,(o.order_units*p.product_unit_cost) AS Actual_total_tobe_paid, \
case \
  when o.Discount_Applied == 'Y' and pay.payment_status == 'paid' then coalesce((p.product_unit_cost-p.product_max_discount_price),p.product_unit_cost)*o.order_units \
  when o.Discount_Applied == 'N' and pay.payment_status == 'paid' then coalesce((p.product_unit_cost-p.product_max_discount_price),p.product_unit_cost)*o.order_units \
  else 0 \
end AS Amount_paid \
from global_temp.orders o \
left outer join global_temp.customer_info c on o.customer_unique_id = c.c_unique_id \
left outer join global_temp.products p on o.product_id = p.product_id \
left outer join global_temp.sellers s on p.product_seller_id = s.seller_id \
left outer join global_temp.payments pay on o.payment_id = pay.payment_transaction_id \
left outer join global_temp.payments_lookup payl on pay.payment_type = payl.payment_type_id").repartition(12).cache()

df_fact_sales_orders.coalesce(1).write.format('csv').option('header','true').mode('overwrite').save('/mnt/salesanalytic/Sales/Transformed_Data/Fact_sales_orders/SparkSQl/')

In [0]:
dbutils.notebook.exit('FFF')